# Topic Modeling using NMF and KLBNMF on Twitter Accounts

In this notebook, we applied a probabilistic matrix decomposition algorithm called NMF and its variation KLBNMF to detect what kind of topics did a specific user tweet about and what are those topics about.


## Natural Language Processing (NLP)

The language of twitter is generally close to daily language. People share their ideas and emotions at any time of the day. Other than normal texts, tweets can include hashtags, emoticons, pictures, videos, gifs, urls etc. Even normal text part of the tweets may consist of misspelled words. Apart from these, one user may tweet in lots of language. For example, one tweet may be in Turkish, and another one in English. So we need to make a cleanup before using those tweets. The list of applied processes:

* Remove Twitter Accounts that has less than 2000 words in their tweets
* Remove URLs
* Tokenization
* Stop words
* Remove non-English words from tweets
* Remove non-English accounts
* Delete accounts whose number of left tokens are less than 200
* Stemming
* Remove words that appears less than 6 timse in the whole corpus

Importing the necessary libraries.

In [1]:
import langid
import logging
import nltk
import numpy as np
import re
import os
import sys
import time
from collections import defaultdict
from string import digits
import pyLDAvis.gensim
import pyLDAvis.sklearn
from gensim import corpora, models, similarities, matutils
import networkx as nx
import string
import math
import pickle

from time import time

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.datasets import fetch_20newsgroups
from sklearn.cluster import KMeans

from collections import Counter

import scipy.io
from scipy import sparse

C:\Users\Burki\Anaconda3\lib\site-packages\gensim\utils.py:855: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\Burki\Anaconda3\lib\site-packages\numpy\lib\utils.py:99: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  warnings.warn(depdoc, DeprecationWarning)


## KLBNMF Codes

In [2]:
# %load gnmf_solvebynewton.py
from __future__ import division
import numpy as np
import scipy as sp
from scipy import special
import numpy.matlib as M

def gnmf_solvebynewton(c, a0 = None):

    if a0 is None:
        a0 = 0.1 * np.ones(np.shape(c))

    M, N = np.shape(a0)
    if len(np.shape(c)) == 0:
        Mc , Nc = 1,1
    else:
        Mc, Nc = np.shape(c)



    a = None
    cond = 0

    if (M == Mc and N == Nc):
        a = a0
        cond = 1

    elif (Mc == 1 and Nc >1):
        cond = 2
        a = a0[0,:]
    elif (Mc > 1 and Nc == 1):
        cond = 3
        a = a0[:,0]
    elif (Mc == 1 and Nc == 1):
        cond = 4
        a = a0[0,0]

    a2 = None
    for index in range(10):
        a2 = a - (np.log(a) - special.polygamma(0,a) + 1 - c) / (1/a - special.polygamma(1,a))
        idx = np.where(a2<0)
        if len(idx[0]) > 0:
            if isinstance(a, float):
                a2 = a / 2
            else:
                a2[idx] = a[idx] / 2
        a = a2

    if(cond == 2):
        a = M.repmat(a,M,1)
    elif(cond == 3):
        a = M.repmat(a,1,N)
    elif(cond == 4):
        a = a * np.ones([M,N])

    return a


In [3]:
# %load gnmf_vb_poisson_mult_fast.py
from __future__ import division
import numpy as np
import scipy as sp
import math
from scipy import special
import numpy.matlib as M

def gnmf_vb_poisson_mult_fast(x,
                            a_tm,
                            b_tm,
                            a_ve,
                            b_ve,
                            EPOCH =1000,
                            Method = 'vb',
                            Update = np.inf,
                            tie_a_ve = 'clamp',
                            tie_b_ve = 'clamp',
                            tie_a_tm = 'clamp',
                            tie_b_tm = 'clamp',
                            print_period = 500
                            ):

    # Result initialiation
    g = dict()
    g['E_T'] = None
    g['E_logT'] = None
    g['E_V'] = None
    g['E_logV'] = None
    g['Bound'] = None
    g['a_ve'] = None
    g['b_ve'] = None
    g['a_tm'] = None
    g['b_tm'] = None

    logm = np.vectorize(math.log)
    W = x.shape[0]
    K = x.shape[1]
    I = b_tm.shape[1]

    M = ~np.isnan(x)
    X = np.zeros(x.shape)
    X[M] = x[M]

    t_init = np.random.gamma(a_tm, b_tm/a_tm)
    v_init = np.random.gamma(a_ve, b_ve/a_ve)
    L_t = t_init
    L_v = v_init
    E_t = t_init
    E_v = v_init
    Sig_t = t_init
    Sig_v = v_init

    B = np.zeros([1,EPOCH])
    gammalnX = special.gammaln(X+1)

    for e in range(1,EPOCH+1):

        LtLv = L_t.dot(L_v)
        tmp = X / (LtLv)
        #check Tranpose
        Sig_t = L_t * (tmp.dot(L_v.T))
        Sig_v = L_v * (L_t.T.dot(tmp))

        alpha_tm = a_tm + Sig_t
        beta_tm = 1/((a_tm/b_tm) + M.dot(E_v.T))
        E_t = alpha_tm * (beta_tm)

        alpha_ve = a_ve + Sig_v
        beta_ve = 1/((a_ve/b_ve) + E_t.T.dot(M))

        E_v = alpha_ve * (beta_ve)
        # Compute the bound
        if(e%10 == 1):
            print("*", end='')
        if(e%print_period == 1 or e == EPOCH):
            g['E_T'] = E_t
            g['E_logT'] = logm(L_t)
            g['E_V'] = E_v
            g['E_logV'] = logm(L_v)

            g['Bound'] = -np.sum(np.sum(M * (g['E_T'].dot(g['E_V'])) + gammalnX))\
                        + np.sum(np.sum(-X * ( ((L_t * g['E_logT']).dot(L_v) + L_t.dot(L_v * g['E_logV']))/(LtLv) - logm(LtLv) ) ))\
                        + np.sum(np.sum((-a_tm/b_tm)* g['E_T'] - special.gammaln(a_tm) + a_tm * logm(a_tm /b_tm)))\
                        + np.sum(np.sum((-a_ve/b_ve)* g['E_V'] - special.gammaln(a_ve) + a_ve * logm(a_ve /b_ve)))\
                        + np.sum(np.sum( special.gammaln(alpha_tm) + alpha_tm * logm(beta_tm) + 1))\
                        + np.sum(np.sum(special.gammaln(alpha_ve) + alpha_ve * logm(beta_ve) + 1 ))

            g['a_ve'] = a_ve
            g['b_ve'] = b_ve
            g['a_tm'] = a_tm
            g['b_tm'] = b_tm

            print()
            print( g['Bound'], a_ve.flatten()[0], b_ve.flatten()[0], a_tm.flatten()[0], b_tm.flatten()[0])
        if (e == EPOCH):
            break;
        L_t = np.exp(special.psi(alpha_tm)) * beta_tm
        L_v = np.exp(special.psi(alpha_ve)) * beta_ve

        Z = None
        if( e> Update):
            if(not tie_a_tm == 'clamp' ):
                Z = (E_t / b_tm) - (logm(L_t) - logm(b_tm))
                if(tie_a_tm == 'clamp'):
                    a_tm = gnmf_solvebynewton(Z,a0=a_tm)
                elif(tie_a_tm == 'rows'):
                    a_tm = gnmf_solvebynewton(np.sum(Z,0)/W, a0=a_tm)
                elif(tie_a_tm == 'cols'):
                    a_tm = gnmf_solvebynewton(np.sum(Z,1)/I, a0=a_tm)
                elif(tie_a_tm == 'tie_all'):
                    #print(np.sum(Z)/(W * I))
                    #print(a_tm)
                    a_tm = gnmf_solvebynewton(np.sum(Z)/(W * I), a0=a_tm)

            if(tie_b_tm == 'free'):
                b_tm = E_t
            elif(tie_b_tm == 'rows'):
                b_tm = M.repmat(np.sum(a_tm * E_t,0)/np.sum(a_tm,0),W,1)
            elif(tie_b_tm == 'cols'):
                b_tm = M.repmat(np.sum(a_tm * E_t,1)/np.sum(a_tm,1),1,I)
            elif(tie_b_tm == 'tie_all'):
                b_tm = (np.sum(a_tm*E_t)/ np.sum(a_tm)) * np.ones([W,I])

            if(not tie_a_ve == 'clamp' ):
                Z = (E_v / b_ve) - (logm(L_v) - logm(b_ve))
                if(tie_a_ve == 'clamp'):
                    a_ve = gnmf_solvebynewton(Z,a_ve)
                elif(tie_a_ve == 'rows'):
                    a_ve = gnmf_solvebynewton(np.sum(Z,0)/I, a0=a_ve)
                elif(tie_a_ve == 'cols'):
                    a_ve = gnmf_solvebynewton(np.sum(Z,1)/K, a0=a_ve)
                elif(tie_a_ve == 'tie_all'):
                    a_ve = gnmf_solvebynewton(np.sum(Z)/(I * K), a0=a_ve)

            if(tie_b_ve == 'free'):
                b_ve = E_v
            elif(tie_b_ve == 'rows'):
                b_ve = M.repmat(np.sum(a_ve * E_v,0)/np.sum(a_ve,0),I,1)
            elif(tie_b_tm == 'cols'):
                b_ve = M.repmat(np.sum(a_ve * E_v,1)/np.sum(a_ve,1),1,K)
            elif(tie_b_tm == 'tie_all'):
                b_ve = (np.sum(a_ve*E_v)/ np.sum(a_ve)) * np.ones([I,K])
    return g


#### Read and Remove Twitter Accounts that has less than 2000 words in their tweets

We have already collected tweets of random 900 followers of TRTWorld's twitter account. You can also find those Twitter API codes in this repo.

Here we are reading each user's tweets from files and saving them into a list (tweetList) if the number of words in the file greater than 2000 words.

In [5]:
tweetsList = []
userList = []

for file in os.listdir("tweets"):
    path = "tweets\\" + file
    f = open(path, 'r', encoding='utf-8')
    fread = f.read()
    if (len(fread.split()) > 2000):
        tweetsList.append(fread)
        userList.append(file[0:len(file)-4])
    f.close()

print(len(tweetsList))
print(len(userList))
print(userList[15])

88
88
15692087


In [39]:
#print(tweetsList[15])

#### Remove URLs

We have removed all urls which are starting with "http://" or "https://. So we excluded all pictures, videos, gifs etc. from the text.

In [6]:
def remove_urls(text):
    text = re.sub(r"(?:\@|http?\://)\S+", "", text)
    text = re.sub(r"(?:\@|https?\://)\S+", "", text)
    return text

def doc_rm_urls():
    return [ remove_urls(tweets) for tweets in tweetsList]

tweetsList = doc_rm_urls()

#print(tweetsList[15])

#### Tokenization
Tokenization is basically process of splitting text into words, phrases or other meaningful elements called tokens. We words as our tokens. To better process the text and to create a dictionary and a corpus we tokenized and converted to lower case all the tweets. We used nltk library with regexp to tokenize. 

In [7]:
# This returns a list of tokens / single words for each user
def tokenize_tweet():
    '''
        Tokenizes the raw text of each document
    '''
    tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
    return [ tokenizer.tokenize(t.lower()) for t in tweetsList]

tweetsList = tokenize_tweet()

#print(tweetsList[15])

#### Stop words
Stop words usually refer to the most common words in a language. So being common makes stopwords less effective and sometimes misleading while making decisions. Thus generally stop words are words which are filtered out. We used nltk library to obtain general English stop words, also we determined some words ourselves and also added one and two character words from tweets to stop words.

In [8]:
# Remove stop words
stoplist_tw=['amp','get','got','hey','hmm','hoo','hop','iep','let','ooo','par',
            'pdt','pln','pst','wha','yep','yer','aest','didn','nzdt','via',
            'one','com','new','like','great','make','top','awesome','best',
            'good','wow','yes','say','yay','would','thanks','thank','going',
            'new','use','should','could','best','really','see','want','nice',
            'while','know']

unigrams = [ t for tweets in tweetsList for t in tweets if len(t)==1]
bigrams  = [ t for tweets in tweetsList for t in tweets if len(t)==2]

stoplist  = set(nltk.corpus.stopwords.words("english") + stoplist_tw + unigrams + bigrams)

tweetsList = [[token for token in tweets if token not in stoplist]
                for tweets in tweetsList]

#print(tokenized_tweets[15])

#### Remove non-English words from tweets
We used nltk corpus to remove non-English words form tweets.

In [9]:
# remove non-english words

words = set(nltk.corpus.words.words())

tweetsList = [[token for token in tweets if token in words or not token.isalpha()]
                for tweets in tweetsList]

#print(tweetsList[558])

#### Remove non-English accounts
It is an extension process to removing non-English words. After removing non-English words from tweets, we removed accounts from our corpus whose tweets are majorly not in English. We used a library called langid to detect English accounts.

In [10]:
# Delete Accounts whose tweets are not majorly in English
print(len(tweetsList))
tweetsList2 = [tweets for tweets in tweetsList if langid.classify(' '.join(tweets))[0] == 'en']
print(len(tweetsList2))

88
86


#### Delete accounts whose number of left tokens are less than 200
After all those preprocessing on tweets, we have removed lots of words from original tweets. Some of the accounts, which are possibly not majorly in English but still includes English words, effected more but still existed in the corpus. So to eliminate those misleading accounts from the corpus we deleted accounts whose number of left tokens are less than 200.

In [11]:
# Delete Accounts whose length of tokenized tweets are less than 200
print(len(tweetsList2))
tweetsList2 = [tweets for tweets in tweetsList2 if len(tweets) > 200]
print(len(tweetsList2))

86
86


#### Remap the User IDs to tweets
In the last two steps we have deleted some accounts. So we need to remap the user ids to the tweets. You can reach tweets from tweetsList2 and users from userList2.

In [12]:
userList2 = []
for i in range(len(tweetsList2)):
    for j in range(i, len(tweetsList)):
        if tweetsList2[i] == tweetsList[j]:
            userList2.append(userList[j])
            break

#### Stemming
For grammatical reasons, documents are going to use different forms of a word, such as organize, organizes, and organizing. Additionally, there are families of derivationally related words with similar meanings, such as democracy, democratic, and democratization. The goal of stemming is to reduce inflectional forms and sometimes derivationally related forms of a word to a common base form. nltk library has mainly 3 kinds of stemming tools for English: lancaster, porter and snowball. We chose Snowball stemmer because it uses a more developed algorithm then Porter Stemmer (Snowball is also called as Porter2) and less aggressive than Lancaster.

In [13]:
# Porter Stemmer and Snowball Stemmer (Porter2) - We useed Snowball Stemmer
# http://stackoverflow.com/questions/10554052/what-are-the-major-differences-and-benefits-of-porter-and-lancaster-stemming-alg

#ps = nltk.stem.PorterStemmer()
#print(ps.stem('I am going'))

sno = nltk.stem.SnowballStemmer('english')

tweetsList2 = [[sno.stem(token) for token in tweets]
          for tweets in tweetsList2]

In [11]:
# Sort words in documents
#for tweets in tokenized_tweets:
#    tweets.sort()

### Dictionary and Corpus

To properly use the Twitter data that we have preprocessed, we need to put into a shape that will be understandable by Topic Modeling algorithms. Bag-of-words representation is perfect fit for those kind of algorithms. In bag-of-words we first created a dictionary which consists of all the words from our preprocessed twitter data as values and their ids as keys. Then we created our corpus. Each element of the corpus corresponds to one Twitter account. Each element consists tuples which includes dictionary id of words and the number of that words' occurrences in that account. We used a very useful python library called Gensim to create our dictionary and corpus.

In [17]:
# Build a dictionary where for each document each word has its own id
dictionary = corpora.Dictionary(tweetsList2)
dictionary.compactify()

print(len(dictionary))

# Build the corpus: vectors with occurence of each word for each document
# convert tokenized documents to vectors
corpus = [dictionary.doc2bow(tweets) for tweets in tweetsList2]

print(len(corpus))

print(dictionary)

25465
86
Dictionary(25465 unique tokens: ['unsung', '11k', '4000m', 'deject', 'n88em']...)


#### Remove words that appears at most 3 times in the whole corpus
This process removes some kind of outlier words (like non-English, meaningless or heavily degenerated words) from the corpus which are passed undetected from the former natural language processes.

In [18]:
# Removing words that appears at most 5 times in the whole corpus

dictCtr = np.zeros(len(dictionary))

for c in corpus:
    for tuples in c:
        dictCtr[tuples[0]] = dictCtr[tuples[0]] + tuples[1]
        
badids = []
for i in range(len(dictCtr)):
    if dictCtr[i] < 6:
        badids.append(i)
        
        
dictionary.filter_tokens(bad_ids=badids)
dictionary.compactify()

corpus = [dictionary.doc2bow(tweets) for tweets in tweetsList2]

print(dictionary)

Dictionary(8346 unique tokens: ['unsung', 'skew', 'empti', 'frank', 'adjust']...)


In [19]:
tweetList = []

for c in corpus:
    str = ''
    for tokens in c:
        str = str + ((dictionary[tokens[0]]+' ') * tokens[1])
    tweetList.append(str)

#print(tweetList[15])
print(len(tweetList))
# tweetList = [' '.join(tweets) for tweets in tokenized_tweets]

86


## sklearn NMF - LDA

### tfidfDense2 = tfidfDense*10000

Sikinti cikmasinin temel sebebi gonderdigimiz matrislerin iclerindeki degerlerin cok kucuk olmasiymis. Gonderdiklerimizin degerlerini 1000 ciavrlarina ciakrinca calismaya basladi. 

In [289]:
n_samples = len(tweetList)
n_features = len(dictionary)
n_topics = 6
n_top_words = 7

def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        sm = sum(topic)
        print("Topic #%d:" % topic_idx)
        for i in topic.argsort()[:-n_top_words - 1:-1]:
            print("(%s, %lf)  " %(feature_names[i], topic[i]/sm), end='')
        print()
    print()
    
def print_top_words2(H, feature_names, n_top_words):
    for topic_idx, topic in enumerate(H):
        sm = sum(topic)
        print("Topic #%d:" % topic_idx)
        for i in topic.argsort()[:-n_top_words - 1:-1]:
            print("(%s, %lf)  " %(feature_names[i], topic[i]/sm), end='')
        print()
    print()

In [290]:
# Use tf-idf features for NMF.
print("Extracting tf-idf features for NMF...")

tfidf_vectorizer = TfidfVectorizer(max_features=n_features)
t0 = time()
tfidf = tfidf_vectorizer.fit_transform(tweetList)
print("done in %0.3fs." % (time() - t0))

Extracting tf-idf features for NMF...
done in 0.670s.


tfidf in 10000 ile carpilmisina KLBNMF uyguluyoruz ve ciktilari klbnmf te tutuyoruz

In [291]:
tfidfDense = tfidf.todense()
tfidfDense2 = tfidfDense*10000
#idx = np.where(tfidfDense2>100)
#print(len(idx[0]))

W = tfidf.shape[0]
K = tfidf.shape[1]
I = 6

a_tm = 1 * np.ones([W,I])
b_tm = np.ones([W,I])
a_ve = np.ones([I,K])
b_ve = 8 * np.ones([I,K])

#T = np.random.gamma(a_tm,b_tm)
#V = np.random.gamma(a_ve,b_ve)

#x = np.random.poisson(T.dot(V))

#idx = np.where(x>100)
#print(len(idx[0]))

klbnmf = gnmf_vb_poisson_mult_fast(np.asarray(tfidfDense2),a_tm,b_tm,a_ve,b_ve,
                                EPOCH=2000,
                                Update =10,
                                tie_a_ve='tie_all',
                                tie_b_ve='tie_all',
                                tie_a_tm='tie_all',
                                tie_b_tm='tie_all')

*
-52888724.7026 1.0 8.0 1.0 1.0
**************************************************
-47799492.8031 0.128607479068 7.99378120922 0.129599021311 0.477578430219
**************************************************
-47799033.8459 0.127867756293 7.99347466748 0.127726400684 0.47759115389
**************************************************
-47799004.4933 0.127738309372 7.99320331253 0.127735011824 0.47759112026
*************************************************
-47798998.8332 0.127713887583 7.99297689383 0.127736920318 0.477590729769


Burasi normal tfidf e NMF uygulayarak sonuc almaca

In [292]:
# Fit the NMF model
print("Fitting the NMF model with tf-idf features, " "n_samples=%d and n_features=%d..." % (n_samples, n_features))
t0 = time()
nmf = NMF(n_components=n_topics, random_state=1, alpha=.1, l1_ratio=.5).fit(tfidf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in NMF model:")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)

#http://nbviewer.jupyter.org/github/bmabey/pyLDAvis/blob/master/notebooks/sklearn.ipynb#topic=0&lambda=1&term=
nmf_vis_data = pyLDAvis.sklearn.prepare(nmf, tfidf, tfidf_vectorizer)
pyLDAvis.display(nmf_vis_data)

Fitting the NMF model with tf-idf features, n_samples=86 and n_features=8346...
done in 0.233s.

Topics in NMF model:
Topic #0:
(work, 0.008980)  (time, 0.008375)  (day, 0.007578)  (today, 0.007094)  (love, 0.006763)  (look, 0.006725)  (learn, 0.005833)  
Topic #1:
(learn, 0.126533)  (data, 0.091379)  (machin, 0.072489)  (deep, 0.050436)  (python, 0.049417)  (neural, 0.031474)  (2010, 0.027980)  
Topic #2:
(robot, 0.312704)  (raspberri, 0.086562)  (video, 0.073137)  (drone, 0.038470)  (kit, 0.034951)  (moss, 0.034431)  (build, 0.031318)  
Topic #3:
(3dprint, 0.416640)  (print, 0.167383)  (3dthursday, 0.050093)  (3dprinter, 0.043315)  (printer, 0.041812)  (lam2017, 0.027934)  (mk2, 0.027571)  
Topic #4:
(stem, 0.200144)  (scienc, 0.083217)  (conasta64, 0.063422)  (learn, 0.059154)  (youth, 0.045197)  (educ, 0.037513)  (today, 0.032519)  
Topic #5:
(analyt, 0.151379)  (data, 0.138156)  (apach, 0.040678)  (busi, 0.040396)  (strategi, 0.038951)  (cloud, 0.029835)  (latest, 0.028192)  



Burasi 10000 ile carptigimiz tfidf e NMF uygulayarak sonuc almaca

In [293]:
# Fit the NMF model
print("Fitting the NMF model with tf-idfDense2 features, " "n_samples=%d and n_features=%d..." % (n_samples, n_features))
t0 = time()
nmf = NMF(n_components=n_topics, random_state=1, alpha=.1, l1_ratio=.5).fit(tfidfDense2)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in NMF model:")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)

tfidfDense2sparse = sparse.csr_matrix(tfidfDense2)

#http://nbviewer.jupyter.org/github/bmabey/pyLDAvis/blob/master/notebooks/sklearn.ipynb#topic=0&lambda=1&term=
nmf_vis_data = pyLDAvis.sklearn.prepare(nmf, tfidfDense2sparse, tfidf_vectorizer)
pyLDAvis.display(nmf_vis_data)

Fitting the NMF model with tf-idfDense2 features, n_samples=86 and n_features=8346...
done in 0.319s.

Topics in NMF model:
Topic #0:
(time, 0.005819)  (work, 0.005394)  (trump, 0.004470)  (day, 0.004189)  (look, 0.004182)  (think, 0.004173)  (love, 0.004071)  
Topic #1:
(learn, 0.018845)  (data, 0.012802)  (machin, 0.009601)  (deep, 0.008360)  (python, 0.007799)  (interest, 0.005658)  (neural, 0.005587)  
Topic #2:
(robot, 0.032871)  (raspberri, 0.010835)  (video, 0.009618)  (drone, 0.006554)  (build, 0.006438)  (kit, 0.006386)  (first, 0.004127)  
Topic #3:
(3dprint, 0.062278)  (print, 0.027352)  (3dthursday, 0.009349)  (printer, 0.008605)  (3dprinter, 0.008174)  (design, 0.006726)  (mk2, 0.006294)  
Topic #4:
(stem, 0.011914)  (learn, 0.011678)  (today, 0.008917)  (maker, 0.008244)  (love, 0.007309)  (scienc, 0.006844)  (day, 0.006362)  
Topic #5:
(analyt, 0.024290)  (data, 0.023989)  (busi, 0.009410)  (learn, 0.009283)  (strategi, 0.007967)  (apach, 0.007088)  (market, 0.006713)  


Burasi klbnmf ten elde ettigimiz matrisleri birbiriyle carparak tum bir asil matrisi olusturup bunu nmf e sokuyoruz. Asil maksat yuvarlakli gorseli elde etmek.

In [294]:
XX = np.dot(klbnmf['E_T'],klbnmf['E_V'])

# Fit the NMF model
print("Fitting the KLBNMF model with tf-idf(klbnmf['E_T']*klbnmf['E_V']) features, " "n_samples=%d and n_features=%d..." % (n_samples, n_features))
t0 = time()
nmf = NMF(n_components=n_topics, random_state=1, alpha=.1, l1_ratio=.5).fit(XX)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in NMF model:")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)

XXsparse = sparse.csr_matrix(XX)

#http://nbviewer.jupyter.org/github/bmabey/pyLDAvis/blob/master/notebooks/sklearn.ipynb#topic=0&lambda=1&term=
nmf_vis_data = pyLDAvis.sklearn.prepare(nmf, XXsparse, tfidf_vectorizer)
pyLDAvis.display(nmf_vis_data)

Fitting the KLBNMF model with tf-idf(klbnmf['E_T']*klbnmf['E_V']) features, n_samples=86 and n_features=8346...
done in 0.288s.

Topics in NMF model:
Topic #0:
(robot, 0.020638)  (raspberri, 0.008837)  (kit, 0.006912)  (video, 0.006500)  (project, 0.005672)  (build, 0.005319)  (check, 0.004639)  
Topic #1:
(data, 0.013971)  (learn, 0.013928)  (machin, 0.006397)  (deep, 0.005747)  (work, 0.005716)  (python, 0.005161)  (interest, 0.004843)  
Topic #2:
(trump, 0.009387)  (time, 0.006389)  (peopl, 0.005227)  (love, 0.004533)  (think, 0.004455)  (work, 0.004136)  (day, 0.004133)  
Topic #3:
(learn, 0.011873)  (stem, 0.009998)  (today, 0.007903)  (love, 0.007885)  (maker, 0.007793)  (day, 0.006371)  (scienc, 0.005840)  
Topic #4:
(analyt, 0.009988)  (data, 0.008354)  (innov, 0.006202)  (busi, 0.005935)  (learn, 0.005738)  (team, 0.005422)  (market, 0.005255)  
Topic #5:
(3dprint, 0.013957)  (print, 0.007923)  (look, 0.005854)  (work, 0.005543)  (time, 0.005198)  (love, 0.004362)  (sorri, 0.0

Asil en anlamli kisim burasi diyebiliriz. Cunku burada direkt klbnmf ten elde ettigimiz decompose edilmis matristeki kelimelerin topiclere nasil dagildigini gozlemleyebiliyoruz.

ilk kisim normal decompose edilmis klbnmf amtrislerin carpiminin nmf i, ikinci kisim direkt klbnmf

Burda klbnmf ile cikan sonuclarin, klbnmf ten cikan matrislerin carpimiyla cikan matrsiten elde ettigimiz sonuclarla neredeyse birebir ayni oldugunu gorebiliriz. Yani yukardaki yuvarlakli guzel gorseli asil olay olarak kullanabiliriz.

In [295]:
print_top_words(nmf, tfidf_vectorizer.get_feature_names(), n_top_words)
print_top_words2(klbnmf['E_V'], tfidf_vectorizer.get_feature_names(), n_top_words)

Topic #0:
(robot, 0.020638)  (raspberri, 0.008837)  (kit, 0.006912)  (video, 0.006500)  (project, 0.005672)  (build, 0.005319)  (check, 0.004639)  
Topic #1:
(data, 0.013971)  (learn, 0.013928)  (machin, 0.006397)  (deep, 0.005747)  (work, 0.005716)  (python, 0.005161)  (interest, 0.004843)  
Topic #2:
(trump, 0.009387)  (time, 0.006389)  (peopl, 0.005227)  (love, 0.004533)  (think, 0.004455)  (work, 0.004136)  (day, 0.004133)  
Topic #3:
(learn, 0.011873)  (stem, 0.009998)  (today, 0.007903)  (love, 0.007885)  (maker, 0.007793)  (day, 0.006371)  (scienc, 0.005840)  
Topic #4:
(analyt, 0.009988)  (data, 0.008354)  (innov, 0.006202)  (busi, 0.005935)  (learn, 0.005738)  (team, 0.005422)  (market, 0.005255)  
Topic #5:
(3dprint, 0.013957)  (print, 0.007923)  (look, 0.005854)  (work, 0.005543)  (time, 0.005198)  (love, 0.004362)  (sorri, 0.004216)  

Topic #0:
(learn, 0.011873)  (stem, 0.009998)  (today, 0.007903)  (love, 0.007885)  (maker, 0.007793)  (day, 0.006371)  (scienc, 0.005840)  